In [5]:
import json
from sseclient import SSEClient as EventSource

url = 'https://stream.wikimedia.org/v2/stream/recentchange'
sampling_rate = 20
counter = 0
N = 40_000
users = {}
with open("edits.json", "w") as file:
    for event in EventSource(url):
        if not event.event == 'message':
            continue
        try:
            change = json.loads(event.data)
        except (ValueError, KeyError):
            continue
            
        if hash(change["meta"]["id"]) % 100 < sampling_rate:
            json.dump(change, file)
            file.write('\n')  # Write each change on a new line
            
            counter += 1
            if counter % 1000 == 0:
                print(f"Processed {counter}/40000 records...")
                
            if counter > N:
                break

Processed 1000/40000 records...

Processed 2000/40000 records...
Processed 3000/40000 records...

Processed 4000/40000 records...
Processed 5000/40000 records...

Processed 6000/40000 records...
Processed 7000/40000 records...
Processed 8000/40000 records...
Processed 9000/40000 records...
Processed 10000/40000 records...
Processed 11000/40000 records...
Processed 12000/40000 records...

Processed 13000/40000 records...
Processed 14000/40000 records...
Processed 15000/40000 records...

Processed 16000/40000 records...
Processed 17000/40000 records...

Processed 18000/40000 records...
Processed 19000/40000 records...
Processed 20000/40000 records...

Processed 21000/40000 records...
Processed 22000/40000 records...

Processed 23000/40000 records...
Processed 24000/40000 records...
Processed 25000/40000 records...

Processed 26000/40000 records...
Processed 27000/40000 records...

Processed 28000/40000 records...
Processed 29000/40000 records...

Processed 30000/40000 records...
Processe